In [1]:
import numpy as np
import json
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import pandas as pd
from generate_dataset import word_set
from gvsm import perform_gvsm

/home/ss/anaconda3/envs/ML_Lab/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Load up dataset
- แต่ละข้อมูลประกอบด้วย
    - documents
    - query
    - รายการค่า cosine similarity ของ document แต่ละตัว กับ query

In [2]:
file_paths = glob.glob('data/dataset_*.json')

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        all_data.append(data)


# Preprocess
- ทำกระบวนการ TFIDF documents กับ query ร่วมกัน ใช้สำหรับเป็น feature
- ใช้ค่า cosine similarity เป็น output
- โดยเราจะทำนายค่า cosine similarity เพื่อใช้สำหรับเรียงลำดับเอกสาร

![tf idf process](./images/tfidf.png)

In [3]:
def preprocess(data):
    i =0
    result = []
    # all doc + query
    doc_strings = [" ".join(doc) for doc in data["documents"]+ [data["query"]]]
    # library tfidf.
    vectorizer = TfidfVectorizer(vocabulary=word_set)
    tfidf_matrix = vectorizer.fit_transform(doc_strings).toarray()

    actual_document = tfidf_matrix[:-1]
    actual_query = tfidf_matrix[-1]

    for x in actual_document:
        mx = list(x)
        for q in actual_query:
            mx.append(q)
        y = data["gvsm_ranked_docs"][i]
        result.append({"input":mx, "output":y})
        i+=1
    return result

# Split train & test
![tf idf process](./images/traintest.png)

In [4]:
X = []
Y = []

for data in all_data:
    result = preprocess(data)
    for r in result:
        X.append(r["input"])
        Y.append(r["output"])

X = np.array(X)
Y = np.array(Y)

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# สร้าง nueral network
![nn](./images/nn.png)

In [5]:

import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    ## fixed input
    layers.Input(shape=(len( X[0] ),)),           # 64 input features
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1,activation='sigmoid') 
])

# mae cuz 0-1
model.compile(optimizer='adam', loss='mae')
model.summary()

2025-04-03 14:43:58.657519: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-03 14:43:58.666133: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-03 14:43:58.675240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-03 14:43:58.677910: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-03 14:43:58.685740: I tensorflow/core/platform/cpu_feature_guar

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,217 (67.25 KB)

 Trainable params: 17,217 (67.25 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# --- Train the Model ---
# Using validation_data so that the test set is evaluated at the end of each epoch
model.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test))

Epoch 1/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 583us/step - loss: 0.1158 - val_loss: 0.0494
Epoch 2/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 380us/step - loss: 0.0469 - val_loss: 0.0484
Epoch 3/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - loss: 0.0437 - val_loss: 0.0501
Epoch 4/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 376us/step - loss: 0.0438 - val_loss: 0.0478
Epoch 5/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 383us/step - loss: 0.0463 - val_loss: 0.0474
Epoch 6/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 394us/step - loss: 0.0442 - val_loss: 0.0481
Epoch 7/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step - loss: 0.0472 - val_loss: 0.0478
Epoch 8/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 375us/step - loss: 0.0463 - val_loss: 0.0483
Epoch 9/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - loss: 0.0436 - val_loss: 0.0483
Epoch 10/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 368us/step - loss: 0.0463 - val_loss: 0.0482
Epoch 11/100
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 394us/step - loss: 0.0457 - val_loss: 0.0475
Epoch 12/100
271/27

In [7]:
# --- Evaluate on the Test Set ---
test_loss = model.evaluate(X_test, Y_test)
print("Test loss:", test_loss)

68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 223us/step - loss: 0.0476
Test loss: 0.04786330461502075


## ทดสอบกับเอกสารจริงเทียบผลระหว่าง gvsm จริงและ nueral network ที่สร้างขึ้นมา

In [8]:
mystuff = {
    "documents":[
        ['bird', 'cat', 'bird', 'cat', 'dog', 'dog', 'bird'],
        ['cat', 'tiger', 'cat', 'dog'],
        ['dog', 'bird', 'bird'],
        ['cat', 'tiger'],
        ['tiger', 'tiger', 'dog', 'tiger', 'cat'],
        ['cat', 'cat', 'tiger', 'tiger'],
        ['bird', 'cat', 'dog'],
        ['dog', 'cat', 'bird'],
        ['cat', 'dog', 'tiger'],
        ['tiger', 'cat', 'tiger'],
        ['bird', 'bird', 'cat', 'dog'],
        ['bird', 'dog', 'dog', 'cat', 'cat', 'tiger'],
        ['cat', 'cat', 'cat', 'cat', 'cat', 'cat'],
        ['dog', 'dog', 'tiger', 'tiger'],
        ['tiger', 'cat', 'tiger', 'cat']
    ],
    "query":["cat","tiger","tiger","tiger"],
    "gvsm_ranked_docs":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]
}

model_input = preprocess(mystuff)
cleanX = []
for x in model_input:
    cleanX.append(x["input"])
predicted_cosim = model.predict(np.array(cleanX))

ranked_docs = pd.DataFrame({
    "Document": [i+1 for i in range(len(predicted_cosim))],
    "CosineSimilarity": [cosim[0] for cosim in predicted_cosim],
}).sort_values(by="CosineSimilarity", ascending=False).reset_index(drop=True)
print("OUR MODEL")
print(ranked_docs)

print("REAL GVSM")
perform_gvsm({str(i+1): doc for i, doc in enumerate(mystuff["documents"])}, mystuff["query"])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
OUR MODEL
    Document  CosineSimilarity
0          5          0.987382
1          4          0.982305
2          6          0.982305
3         15          0.982305
4         10          0.981129
5          2          0.980599
6          9          0.976181
7         14          0.963009
8         12          0.962110
9         13          0.831842
10         7          0.810230
11         8          0.810230
12         1          0.806714
13        11          0.804221
14         3          0.783577
REAL GVSM

Ranked Documents by Cosine Similarity:
    Document  CosineSimilarity  Minterm
0        10          0.998270        4
1         6          0.984094        4
2         4          0.984094        4
3        15          0.984094        4
4         5          0.978355        2
5         9          0.890254        2
6         2          0.889459        2
7        14          0.875229        7
8        13          0.865290        6
9        12    

/home/ss/repos/PYTHON/KMITL/websearch/gvsm.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[[0.66666667]]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tf_query[freq_query != 0] = 0.5 + (0.5 * freq_query[freq_query != 0] / max_freq_query)
/home/ss/repos/PYTHON/KMITL/websearch/gvsm.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3979400086720376' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tf_idf_documents.loc[index, term] = tf_idf_documents.loc[index, term]/maxfreq_by_documents[index] * idf_query[term]
/home/ss/repos/PYTHON/KMITL/websearch/gvsm.py:66: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.04143193783256295' has dtype incompatible wi

array([0.49005675, 0.88945854, 0.26278893, 0.98409429, 0.97835496,
       0.98409429, 0.54302257, 0.54302257, 0.89025365, 0.99827029,
       0.44904658, 0.73919777, 0.86529011, 0.87522865, 0.98409429])